##Import

In [ ]:
!pip install shap

import shap
import pandas as pd
import numpy as np
from glob import glob
import re
import json
import csv
import os
import tensorflow as tf
# tf.compat.v1.disable_v2_behavior()
import io

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential

from keras import Sequential
from keras.layers import Dense, Dropout
tf.config.experimental_run_functions_eagerly(True)

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

import matplotlib.pyplot as plt

from google.colab import drive

     |████████████████████████████████| 371 kB 5.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for shap: filename=shap-0.40.0-cp37-cp37m-linux_x86_64.whl size=509253 sha256=1b90246f934c0bb16c6c957019db84d57c2ae67c94a78faa37c073ec2d6c18fb
  Stored in directory: /root/.cache/pip/wheels/ec/35/84/e304841ac4b910bc95fe9a6e5302eb2507b4333728851dcbfb
Successfully built shap
Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
english_stopwords = stopwords.words("english") #From ICHCL Baseline from Hasoc
with open('/content/drive/MyDrive/ICON/final_stopwords.txt', encoding = 'utf-8') as f:
    hindi_stopwords = f.readlines()
    for i in range(len(hindi_stopwords)):
        hindi_stopwords[i] = re.sub('\n','',hindi_stopwords[i])
stopwords = english_stopwords + hindi_stopwords
english_stemmer = SnowballStemmer("english")

In [ ]:
with open('/content/drive/MyDrive/ICON/bengali_sw.txt', encoding = 'utf-8') as f:
    ben_stopwords = f.readlines()
    for i in range(len(ben_stopwords)):
        ben_stopwords[i] = re.sub('\n','',ben_stopwords[i])
stopwords = stopwords + ben_stopwords

In [ ]:
words_dict  = { "तैराक":"तैर",
                "चालाक":"चाल",
                "कूलाक":"कूल",
                "बेलन":"बेल",
                "मिलाप":"मिल",
                "चुपचाप": "चुप",
                "निकास":"निकस",
                "लुकास":"लुक",
                }

In [ ]:
suffixes = {
	    1: ["ो", "े", "ू", "ु", "ी", "ि", "ा"],  
            2: ["तृ","ान","ैत","ने","ाऊ","ाव","कर", "ाओ", "िए", "ाई", "ाए", "नी", "ना", "ते", "ीं", "ती",
                "ता", "ाँ", "ां", "ों", "ें","ीय", "ति","या", "पन", "पा","ित","ीन","लु","यत","वट","लू"],     
            3: ["ेरा","त्व","नीय","ौनी","ौवल","ौती","ौता","ापा","वास","हास","काल","पान","न्त","ौना","सार","पोश","नाक",
                "ियल","ैया", "ौटी","ावा","ाहट","िया","हार", "ाकर", "ाइए", "ाईं", "ाया", "ेगी", "वान", "बीन",
                "ेगा", "ोगी", "ोगे", "ाने", "ाना", "ाते", "ाती", "ाता", "तीं", "ाओं", "ाएं", "ुओं", "ुएं", "ुआं","कला","िमा","कार",
                "गार", "दान","खोर"],     
            4: ["ावास","कलाप","हारा","तव्य","वैया", "वाला", "ाएगी", "ाएगा", "ाओगी", "ाओगे", 
                "एंगी", "ेंगी", "एंगे", "ेंगे", "ूंगी", "ूंगा", "ातीं", "नाओं", "नाएं", "ताओं", "ताएं", "ियाँ", "ियों", "ियां",
                "त्वा","तव्य","कल्प","िष्ठ","जादा","क्कड़"],     
            5: ["ाएंगी", "ाएंगे", "ाऊंगी", "ाऊंगा", "ाइयाँ", "ाइयों", "ाइयां", "अक्कड़","तव्य:","निष्ठ"],
}

special_suffixes = ["र्", "ज्य","त्य"]
dict_special_suffixes = {"र्":"ृ",
                         "ज्य":"ज्",
                         "त्य":"त्"}

In [ ]:
def hi_stem(word, clean=False,chars=None):
    if clean == True:
        word = clean_text(word, chars)
    
    ans = word
    bl = False
    
    if word in words_dict.keys():
        return words_dict[word]
    
    for L in 5, 4, 3, 2, 1:
        if len(word) > L + 1:
            for suf in suffixes[L]:
                if word.endswith(suf):
                    ans = word[:-L]
                    bl =True
        if bl == True:
            break
                    
    if bl == True:
        for suf in suffixes[1]:
            if ans.endswith(suf):
                # use case - गानेवाला
                ans = hi_stem(ans)
   
    for suf in special_suffixes:
        if ans.endswith(suf):
            l = len(suf)
            ans = ans[:-l]
            ans += dict_special_suffixes[suf]
 
    return ans

In [ ]:
def clean_text(text, chars=None):
    if chars == None:        
        text = re.sub(r"[()\"#/@;:<>{}`+=~|!?,']", "", text)
    else:
        text = re.sub(r"[" +chars+ "()\"#/@;:<>{}`+=~|!?,']", "", text)
    return text

## Read Data

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/ICON/Training Data/train_multi.tsv', delimiter='\t')

In [ ]:
test_data = pd.read_csv('/content/drive/MyDrive/ICON/Training Data/dev_multi.tsv', delimiter='\t')

In [ ]:
# data = pd.concat([data,dev_data])

In [ ]:
# test_data =  pd.read_csv('/content/drive/MyDrive/ICON/test_multi.tsv', delimiter='\t')

In [ ]:
data.head()

,ID,Text,Labels
0,C575.439,इनके बिडीओ जनता के सामने जरूर आने चाहिए,"(OAG,NGEN,NCOM)"
1,C130.179.1,Yam chumme ekaiba khngde mashagi prestige su a...,"(CAG,NGEN,NCOM)"
2,facebook_corpus_msr_1563601,Matt Ellis I would not believe a thing israel ...,"(OAG,NGEN,NCOM)"
3,facebook_corpus_msr_233885,"राजनिति कब तक होती रहेगी, ये सब कब तक चलता रहे...","(CAG,NGEN,NCOM)"
4,1278685975,pahali bara muphtakhori se mukta bajata. #supe...,"(CAG,NGEN,NCOM)"


In [ ]:
test_data.head()

,ID,Text,Labels
0,C95.115,Sir Syed Ahmed se udba kuire ymmmm nungcyeeee,"(NAG,NGEN,NCOM)"
1,C12.55,"Mapanda lairik tamba chatlise yu thakpa, party...","(CAG,NGEN,NCOM)"
2,C100.39.2,Keidano pktrise nangonda hyrk Adi nungygadro n...,"(OAG,NGEN,NCOM)"
3,C101.919.1,Tiken kangyet kangyet lairik hieric Manipur ge...,"(CAG,NGEN,NCOM)"
4,C1035.324.2,@Big monster tor himmot ache marar tmc holo bj...,"(OAG,NGEN,NCOM)"


In [ ]:
data['Labels'].value_counts()

(NAG,NGEN,NCOM)    2672
(OAG,NGEN,NCOM)    1677
(CAG,NGEN,NCOM)    1547
(OAG,GEN,NCOM)     1351
(OAG,NGEN,COM)      932
(CAG,GEN,NCOM)      291
(CAG,NGEN,COM)      289
(NAG,GEN,NCOM)      182
(OAG,GEN,COM)       136
(NAG,NGEN,COM)       98
(CAG,GEN,COM)        32
(NAG,GEN,COM)         7
Name: Labels, dtype: int64

##Preprocessing

In [ ]:
regex_for_english_hindi_emojis="[^a-zA-Z#\U0001F300-\U0001F5FF'|'\U0001F600-\U0001F64F'|'\U0001F680-\U0001F6FF'|'\u2600-\u26FF\u2700-\u27BF\u0900-\u097F]"
def clean_text(text):
    text = re.sub(r"@[A-Za-z0-9]+",' ', text)
    text = re.sub(r"https?://[A-Za-z0-9./]+",' ', text)
    text = re.sub(regex_for_english_hindi_emojis,' ', text)
    text = re.sub("RT ", " ", text)
    text = re.sub("\n", " ", text)
    text = re.sub(r" +", " ", text)
    tokens = []
    for token in text.split():
        if token not in stopwords:
            token = english_stemmer.stem(token)
            token = hi_stem(token)
            tokens.append(token)
    return " ".join(tokens)

def convert_gen(text):
    label = text[1:-1].split(",")
    text = label[1]
    if text == "GEN":
      return 1
    return 0

def convert_com(text):
    label = text[1:-1].split(",")
    text = label[2]
    if text == "COM":
      return 1
    return 0

def convert_agg(text):
    label = text[1:-1].split(",")
    text = label[0]
    if text == "OAG":
      return 2
    if text == "CAG":
      return 1
    return 0

In [ ]:
cleaned_text = [clean_text(text) for text in data.Text]
test_cleaned_text = [clean_text(text) for text in test_data.Text]

In [ ]:
cleaned_text

['बिडीओ जन के साम जरूर आन',
 'yam chumm ekaiba khngde mashagi prestig su amtta yengdr touris karino nupigi lanbu thengnagadra leibakki potlumbu pugadra khara leiragadi nupas mental ill oikhidana sarkta thoklk adiii oiiimaahh komaaiii twni mee machana twgadava thabk ni sidi',
 'matt elli i would believ thing israel say israel lie like sugar daddi america it claim countri care look behind curtain noth lie hatr toward other death',
 'राजन रह सब चल रह कड निन्द ओर श्रद्धांजल समाध कारक उपाय करिय',
 'pahali bara muphtakhori se mukta bajata #superbudget #sabkabudget #unionbudget',
 'rrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrurrrrrrrrrrrrrrrryrrrrr',
 'delhi per dhayan',
 'ym thina wabini pkte haibas crime ntte',
 'right abba bjp er dalal ekta vote kete bjp ke jetat asech eke ekta vote noi',
 'jai ram',
 'oja tiken kanglada mandop sabei matangdo ningsinglakeda',
 'hell bjp btw bjp spokesperson tiken bijoy da mayamna thina ngangbasi toksi kanad moi ani fataba mi ngakta su nate inc 

In [ ]:
y_agg = data.Labels.apply(lambda x: convert_agg(x))
# y_test_agg = test_data.Labels.apply(lambda x: convert_agg(x))
#NAG - 0; CAG - 1; OAG - 2

In [ ]:
y_gen = data.Labels.apply(lambda x: convert_gen(x))
# y_test_gen = test_data.Labels.apply(lambda x: convert_gen(x))
#NGEN - 0; GEN - 1

In [ ]:
y_com = data.Labels.apply(lambda x: convert_com(x))
# y_test_com = test_data.Labels.apply(lambda x: convert_com(x))
#NCOM - 0; COM - 1

## Bidirectional LSTM Model

In [ ]:
top_words = 10000
tokenizer = Tokenizer(num_words=top_words)
tokenizer.fit_on_texts(cleaned_text)
word_index = tokenizer.word_index
list_tokenized_train = tokenizer.texts_to_sequences(cleaned_text)

max_review_length = 130
embedding_vecor_length = 32
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

###Gender Bias

In [ ]:
X_train = pad_sequences(list_tokenized_train, maxlen=max_review_length)
y_train = y_gen

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(top_words+1, embedding_vecor_length, input_length=max_review_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# model = tf.keras.models.Sequential()
# model.add(tf.keras.layers.Embedding(top_words+1, embedding_vecor_length, input_length=max_review_length))
# model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))
# model.add( tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)))
# model.add( tf.keras.layers.Dense(64, activation='relu'))
# model.add( tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 130, 32)           320032    
                                                                 
 bidirectional (Bidirectiona  (None, 130, 128)         49664     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 415,137
Trainable params: 415,137
Non-trai

In [ ]:
history = model.fit(X_train,y_train, epochs=20, batch_size=64)

Epoch 1/20
  1/191 [..............................] - ETA: 24s - loss: 0.1644 - accuracy: 0.9375

Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.


191/191 [==============================] - 25s 131ms/step - loss: 0.1981 - accuracy: 0.9188
Epoch 2/20
191/191 [==============================] - 25s 131ms/step - loss: 0.1467 - accuracy: 0.9363
Epoch 3/20
191/191 [==============================] - 25s 131ms/step - loss: 0.1183 - accuracy: 0.9475
Epoch 4/20
191/191 [==============================] - 25s 131ms/step - loss: 0.1067 - accuracy: 0.9507
Epoch 5/20
191/191 [==============================] - 25s 130ms/step - loss: 0.1013 - accuracy: 0.9533
Epoch 6/20
191/191 [==============================] - 25s 130ms/step - loss: 0.1005 - accuracy: 0.9532
Epoch 7/20
191/191 [==============================] - 25s 131ms/step - loss: 0.1016 - accuracy: 0.9514
Epoch 8/20
191/191 [==============================] - 25s 130ms/step - loss: 0.0951 - accuracy: 0.9535
Epoch 9/20
191/191 [==============================] - 25s 131ms/step - loss: 0.0938 - accuracy: 0.9548
Epoch 10/20
191/191 [==============================] - 25s 131ms/step - loss: 0.0923

In [ ]:
list_tokenized_test = tokenizer.texts_to_sequences(test_cleaned_text)
X_test = pad_sequences(list_tokenized_test, maxlen=max_review_length)
prediction = model.predict(X_test)
y_pred_gen = (prediction > 0.5)

e = model.layers[0]
weights = e.get_weights()[0]

Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.


In [ ]:
out_v = io.open('vecs_gender.tsv', 'w', encoding='utf-8')
out_m = io.open('meta_gender.tsv', 'w', encoding='utf-8')
for word_num in range(1, top_words):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [ ]:
print(len(X_train[0]))
print(X_train[0])

130
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0  527
   12 1271 1854 2360]


In [ ]:
print(len(X_test[0]))
print(X_test[0])

130
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    7 3945 3518 1369   13   58 6851   25    1 2623 2742   20   16
 8085  231    7    1]


In [ ]:
# attrib_data = X_train[:200]
# explainer = shap.DeepExplainer(model, attrib_data)
# num_explanations = 20
# shap_vals = explainer.shap_values(X_test[:num_explanations])
# words = processor._tokenizer.word_index
# word_lookup = list()
# for i in words.keys():
#   word_lookup.append(i)
# word_lookup = [''] + word_lookup
# shap.summary_plot(shap_vals, feature_names=word_lookup, class_names=tag_encoder.classes_)

Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


AttributeError: ignored

In [ ]:
# try:
#   from google.colab import files
# except ImportError:
#   pass
# else:
#   files.download('vecs_gender.tsv')
#   files.download('meta_gender.tsv')

In [ ]:
# print("Accuracy of the model : ", accuracy_score(y_pred_gen, y_test_gen))
# print('F1-score: ', f1_score(y_pred_gen, y_test_gen))
# print('Confusion matrix:')
# confusion_matrix(y_test_gen,y_pred_gen)

In [ ]:
# print(classification_report(y_test_gen,y_pred_gen))

### Communal Bias

In [ ]:
y_train = y_com

NameError: ignored

In [ ]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(top_words+1, embedding_vecor_length, input_length=max_review_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model2.summary()

In [ ]:
history2 = model2.fit(X_train,y_train, epochs=20, batch_size=64)

In [ ]:
prediction = model2.predict(X_test)
y_pred_com = (prediction > 0.5)


e = model2.layers[0]
weights = e.get_weights()[0]

In [ ]:
out_v = io.open('vecs_comm.tsv', 'w', encoding='utf-8')
out_m = io.open('meta_comm.tsv', 'w', encoding='utf-8')
for word_num in range(1, top_words):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [ ]:
attrib_data = X_train[:2000]
explainer = shap.DeepExplainer(model2, attrib_data)
num_explanations = 20
shap_vals = explainer.shap_values(X_test[:num_explanations])
words = processor._tokenizer.word_index
word_lookup = list()
for i in words.keys():
  word_lookup.append(i)
word_lookup = [''] + word_lookup
shap.summary_plot(shap_vals, feature_names=word_lookup, class_names=tag_encoder.classes_)

In [ ]:
# try:
#   from google.colab import files
# except ImportError:
#   pass
# else:
#   files.download('vecs_comm.tsv')
#   files.download('meta_comm.tsv')

In [ ]:
# print("Accuracy of the model : ", accuracy_score(y_pred_com, y_test_com))
# print('F1-score: ', f1_score(y_pred_com, y_test_com))
# print('Confusion matrix:')
# confusion_matrix(y_test_com,y_pred_com)

In [ ]:
# print(classification_report(y_test_com,y_pred_com))

### Aggressiveness

In [ ]:
X_train = pad_sequences(list_tokenized_train, maxlen=max_review_length)
y_train = y_agg

In [ ]:
list_tokenized_test = tokenizer.texts_to_sequences(test_cleaned_text)
X_test = pad_sequences(list_tokenized_test, maxlen=max_review_length)
# prediction = model.predict(X_test)
# y_pred_gen = (prediction > 0.5)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(top_words+1, embedding_vecor_length, input_length=max_review_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])
model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 130, 32)           320032    
                                                                 
 bidirectional (Bidirectiona  (None, 130, 128)         49664     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 3)                 195       
                                                                 
Total params: 415,267
Trainable params: 415,267
Non-trai

In [ ]:
history = model.fit(X_train,y_train, epochs=20, batch_size=64)

Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.


Epoch 1/20
144/144 [==============================] - 23s 128ms/step - loss: 0.9888 - accuracy: 0.5125
Epoch 2/20
144/144 [==============================] - 18s 127ms/step - loss: 0.7209 - accuracy: 0.6904
Epoch 3/20
144/144 [==============================] - 18s 127ms/step - loss: 0.5135 - accuracy: 0.7930
Epoch 4/20
144/144 [==============================] - 18s 127ms/step - loss: 0.3595 - accuracy: 0.8565
Epoch 5/20
144/144 [==============================] - 18s 127ms/step - loss: 0.2789 - accuracy: 0.8897
Epoch 6/20
144/144 [==============================] - 18s 127ms/step - loss: 0.2280 - accuracy: 0.9079
Epoch 7/20
144/144 [==============================] - 18s 126ms/step - loss: 0.1984 - accuracy: 0.9189
Epoch 8/20
144/144 [==============================] - 18s 126ms/step - loss: 0.1747 - accuracy: 0.9250
Epoch 9/20
144/144 [==============================] - 18s 126ms/step - loss: 0.1696 - accuracy: 0.9245
Epoch 10/20
144/144 [==============================] - 18s 126ms/step - l

In [ ]:
y_test_agg = test_data.Labels.apply(lambda x: convert_agg(x))

In [ ]:
prediction = model.predict(X_test)

Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.


In [ ]:
prediction 

array([[9.9999154e-01, 2.4686660e-06, 5.9176182e-06],
       [4.5156521e-05, 2.7362890e-05, 9.9992752e-01],
       [8.7503800e-03, 1.3144379e-03, 9.8993522e-01],
       ...,
       [9.9998486e-01, 7.6206943e-06, 7.5620674e-06],
       [1.9774271e-02, 1.3996847e-03, 9.7882611e-01],
       [9.9999809e-01, 4.2444015e-07, 1.4417325e-06]], dtype=float32)

In [ ]:
pred_list = []
for i in range(len(prediction)):
  pred_list.append(list(prediction[i]))

In [ ]:
for i in range(len(prediction)):
  pred_list[i] = pred_list[i].index(max(pred_list[i]))


In [ ]:
y_pred_agg = pd.Series(pred_list)

In [ ]:
print("Accuracy of the model : ", accuracy_score(y_pred_agg, y_test_agg))
# print('F1-score: ', f1_score(y_pred_agg, y_test_agg))
print('Confusion matrix:')
confusion_matrix(y_test_agg,y_pred_agg)

Accuracy of the model :  0.5679012345679012
Confusion matrix:


array([[668, 182, 157],
       [294, 320, 183],
       [280, 199, 714]])

In [ ]:
print(classification_report(y_test_agg,y_pred_agg))

              precision    recall  f1-score   support

           0       0.54      0.66      0.59      1007
           1       0.46      0.40      0.43       797
           2       0.68      0.60      0.64      1193

    accuracy                           0.57      2997
   macro avg       0.56      0.55      0.55      2997
weighted avg       0.57      0.57      0.57      2997



In [ ]:
vectorizer = TfidfVectorizer(min_df = 5)
X = vectorizer.fit_transform(cleaned_text)
X = X.todense()

In [ ]:
X_test = vectorizer.transform(test_cleaned_text)
X_test = X_test.todense()

In [ ]:
X_train = X
y_train = y_agg
y_test = y_test_agg

In [ ]:
lr_agg = LogisticRegression(multi_class='multinomial',max_iter=1000)
lr_agg.fit(X_train, y_train)

np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html


LogisticRegression(max_iter=1000, multi_class='multinomial')

In [ ]:
y_pred_agg = lr_agg.predict(X_test)

np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html


In [ ]:
print("Accuracy of the model : ", accuracy_score(y_pred_agg, y_test_agg))
# print('F1-score: ', f1_score(y_pred_agg, y_test_agg))
print('Confusion matrix:')
confusion_matrix(y_test_agg,y_pred_agg)

Accuracy of the model :  0.6046046046046046
Confusion matrix:


array([[705, 123, 179],
       [275, 326, 196],
       [286, 126, 781]])

In [ ]:
print(classification_report(y_test_agg,y_pred_agg))

              precision    recall  f1-score   support

           0       0.56      0.70      0.62      1007
           1       0.57      0.41      0.48       797
           2       0.68      0.65      0.66      1193

    accuracy                           0.60      2997
   macro avg       0.60      0.59      0.59      2997
weighted avg       0.61      0.60      0.60      2997



##Submission

In [ ]:
def write_preds(input, preds, fname, delim='\t'):
    with open (fname, 'w') as f_w:
        writer = csv.writer(f_w, delimiter=delim)
        writer.writerow(['ID', 'Labels'])
        for id, label in zip(input, preds):
            writer.writerow([id, label])

In [ ]:
isSaveLR = True

In [ ]:
submission = []
a = [0 for i in y_pred_agg]
b = [0 for i in y_pred_agg]
c = [0 for i in y_pred_agg]
for i in range(len(y_pred_agg)):
  if y_pred_agg[i] == 0:
    a[i] = "NAG"
  elif y_pred_agg[i] == 1:
    a[i] = "CAG"
  else:
    a[i] = "OAG"

  if y_pred_gen[i] == 0:
    b[i] = "NGEN"
  else:
    b[i] = "GEN"

  if y_pred_com[i] == 0:
    c[i] = "NCOM"
  else:
    c[i] = "COM"

  string = str((a[i],b[i],c[i]))
  newString = ""
  for j in string:
    if j == "'" or j == ' ':
      continue
    else:
      newString += j
  submission.append(newString)

In [ ]:
submission

In [ ]:
 ids = list(test_data['ID'])

In [ ]:
if not os.path.exists('ens_submission3'):
  os.makedirs('ens_submission3')
if isSaveLR:
  write_preds(ids,submission,'ens_submission3/pred_multi.tsv')